<h1>4" Stacker Servo Integration</h1>
<h3>20201RFP_1</h3><br>

<h2>Objective</h2>
<p>The overall concept is to drive the stacker wheel with a servo that can accomplish these movements</p>
<div>
  <ul>
    <li>Accelerate from start to 2.5 in/s linear velocity</li>
    <li>Stop load within .1s when bad pads are detected</li>
    <li>Restart load within .1s when pads become good again </li>
    <li>Every 30 pads, skip two by accelerting and returing to 2.5 in/s within in .2s </li>
    <li>Keep an even pace at 2.5 in/s the rest of the time</li>
    <li>Be able to stop on machine shutdown.</li>
  </ul>
</div><br>

<h2>Summary of stacker</h2>

<p>Stacker's function is to collect pads from the line, which are produced at 300rps (or 1 pad every .2 seconds), and then feed them out of the bagging funnel in groups of 30 for packaging.  Pads are fed one at a time into the wheel and then round over the top of the stacker where they are brought down in a straight line.  An ejection rod which is actuated by air cylinder then fires at precisly the right time to grap 30 pads from the wheel.  This timing window has a maximum vaience of .1ms (or 4mm), which once excceeded results in clipping an extra pad and potentially jamming the machine.  With the servo movements listed above, we intend to speed the wheel up and put a two paddle space between each 30th pad.  This will triavalize the timing needed from the aircyliders to make the system work. </p>

To size the needed servo we will need several things:
<ul>
  <li>Linear load on stacker</li>
  <li>Inertia of the system</li>
  <li>Motion parameters for linear movement on stacker</li>
  <li>Rotational motion parameters for server drive</li>
</ul>

<h2>Linear load on stacker</h2>

<img src="./assets/ChainSystemBasic.svg">

<p>The stacker consists of 3 double sprockets (1 driven, 2 idle), two 40 chain with ears (1/2" pitch), plastic paddles that hold the product, and the product itsself.  Dimensions of these are as follows:</p>
<ul>
  <li>
    <b>Chain</b>
    <ul>
      <li>75 links</li>
      <li>Size 40 (1/2" pitch)</li>
      <li>0.071666667 lbs/in</li>
      <li>75 in</li>
      <li>2 counts</li>
    </ul>
  </li>
  <li>
    <b>Paddles</b>
    <ul>
      <li>60 grams</li>
      <li>150 counts</li>
    </ul>
  </li>
  <li>
    <b>Product In Wheels</b>
    <ul>
      <li>23.2 grams</li>
      <li>2/3 of paddles maximum full of product (100 counts)</li>
    </ul>
  </li>
  <li>
    
  </li>
</ul>

In [99]:
import helpers
_ = helpers.DisplayHelper(globals())

import pint
ureg = pint.UnitRegistry()

#Calculate linear per link mass:
chainMassPerInch = .071666667 * (ureg.lbs / ureg.inch)
chainMassPerInch = chainMassPerInch.to('kilograms/inch')

paddleMassPerInch = 60 * 2 * (ureg.grams / ureg.inch)
paddleMassPerInch = paddleMassPerInch.to('kilograms/inch')

productMassPerInch = 23.2 * 2 * (ureg.grams / ureg.inch)
productMassPerInch = productMassPerInch.to('kilograms/inch')

print(_('chainMassPerInch','paddleMassPerInch','productMassPerInch'))

chainMassPerInch = 0.0325074533345308 kilogram / inch
paddleMassPerInch = 0.12 kilogram / inch
productMassPerInch = 0.0464 kilogram / inch


<h2>Now we need to calculate the linear force of each side</h2>

<p> we can simply trig out the system if compare it to a triangle, which will make it easy to diagram </p>
<img src="./assets/ChainSystemTrigOverlay.svg" >

<p> Now lets simplify by dropping the actual system and sticking with a triangle</p>

<img src="./assets/ChainSystemTrig.svg" > <br>

<i><u>Metrics</u></i><br>
Side<sub>a</sub> = 28"

<i><u>Calcs</u></i><br>
O<sub>top</sub> = Sin(43<sup>o</sup>) * Side<sub>a</sub> <br>
Side<sub>d</sub> = Cos(43<sup>o</sup>) * Side<sub>a</sub> <br> 
$Side_b = \frac{Cos(43_o) * Side_a}{Cos(21_o)}$ <br>
O<sub>bottom</sub> = Tan(21<sup>o</sup>) * Cos(43<sup>o</sup>) * Side<sub>a</sub> <br>
Side<sub>c</sub> = O<sub>top</sub> + O<sub>bottom</sub>

In [100]:
from math import sin, cos, tan, radians

#Metrics
Side_a = 28 * ureg.inches

#Calcs
O_top = sin(radians(43)) * Side_a
Side_d = cos(radians(43)) * Side_a
Side_b = (cos(radians(43)) * Side_a) / cos(radians(21))
O_bottom = tan(radians(21)) * cos(radians(43)) * Side_a
Side_c = O_top + O_bottom

print(_("O_top", "Side_d", "Side_b", "O_bottom", "Side_c"))

O_top = 19.09595408174996 inch
Side_d = 20.477903645336774 inch
Side_b = 21.93480396988395 inch
O_bottom = 7.860730722365425 inch
Side_c = 26.956684804115383 inch


<p>With all of our sides and angles tigged out we can now develop the force diagrams</p>

<img src="./assets/ChainSystemForces.svg">

<p>We will have to look at each of the force directions seperatly, but first we must look at our motion profile and determine our max acceleration and fill in the rest of our motion profile.</p>

<h2>Motion Profile</h2>

<p>As a reminder here move we need to make<p>
<div>
  <ul>
    <li>Accelerate from start to 2.5 in/s linear velocity</li>
    <li>Stop load within .1s when bad pads are detected</li>
    <li>Restart load within .1s when pads become good again </li>
    <li>Every 30 pads, skip two by accelerting and returing to 2.5 in/s within in .2s </li>
    <li>Keep an even pace at 2.5 in/s the rest of the time</li>
    <li>Be able to stop on machine shutdown.</li>
  </ul>
</div><br>

<p>Here is a velocity profile showing this:</p>
<img src="./assets/ChainSystemMotionProfile.svg">




<h3><b><u>Global Calculations</u></b></h3>

<img src="./assets/PaddleSideViewAnnotated.svg">

The verticle height of one stacker paddle is .48".  This is due to the fact that the pitch of the chain which it mounts to is .50".  Thus, our smallest increment of distance is .50" in our calculations.  Our target goal is always to run at 300 pads per minute.  From here we can now our target constant velocity while runnning:

<i><u>Calcs</u></i><br>
Seconds Per Pad = $\frac{60s}{300}$ = .2s <br>
V<sub>run</sub> = $\frac{.5"}{.2s}$ = 2.5<sup>"/s</sup><br>

In [101]:
#Calcs
SecondsPerPad = (60 * ureg.seconds) / 300
V_run = (.5 * ureg.inches) / SecondsPerPad

g = 9.81 * (ureg.meters / ureg.seconds**2)
g = g.to('inches/second**2')

F_friction = 0

print(_('SecondsPerPad', 'V_run'))

SecondsPerPad = 0.2 second
V_run = 2.5 inch / second


<h2><u>Gavitational total force caclulations</u></h2>

<p>We now need to individually consider our forces to calculate load.</p>

<img src="./assets/ChainSystemForces.svg">

<p> Pads are loaded into the stacker/triagle at the apex of the 64<sup>o</sup> vertex.  Along the side of the triangle marked 'a' the wheel is always full of product.  As the product rounds the 47<sup>o</sup> vertex, and travels directly downward, anong side 'c', the product is ejected in a block of thirty.  Since the pads are moving along with gravity on c, the maximum load occurs after the pads are ejected and their weight can no longer assist the motor.  This fact is hower reveresed when trying to slow the motor, so the calculation will have to be done both ways.  Lastly, the empty paddles round the 69<sub>o</sub> vertex and are always empty.  Thus, no product weight need be included.

To summeraize we have two resting loads:
<ul>
  <li>Side<sub>c</sub> full of pads (prior to ejection)</li>
  <li>Side<sub>c</sub> empty of pads</li>
</ul>

<p>The calcs for either scenerio will be the same, but metrics will change:</p>
<i><u>Metrics</u></i><br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c_scenerio1} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c$ <br>
$m_{side\_c_scenerio2} = (chainMassPerInch + paddleMassPerInch) * Side_c$ <br>

<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{ag} = m_{side\_a}gCos(43^o)$<br>
$F_{bg} = m_{side\_b}gCos(180^o - 69^o)$<br>
$F_{cg} = -m_{side\_c}g + m_{side\_c}$<br>
$F_{g}$ = F_{ag} + F_{bg} + F_{cg}$

In [102]:
#Metrics
m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c_wProduct = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c
m_side_c_nProduct = (chainMassPerInch + paddleMassPerInch) * Side_c

#Calcs
F_ag = -m_side_a * g * cos(radians(43))
F_bg = -m_side_b * g * abs(cos(radians(180 - 43)))
F_cg_wProduct = m_side_c_wProduct * g
F_cg_nProduct = m_side_c_nProduct * g
F_g_wProduct = F_ag + F_bg + F_cg_wProduct
F_g_nProduct = F_ag + F_bg + F_cg_nProduct

#Normalize
F_g_wProduct = (F_g_wProduct.to('newtons'))
F_g_nProduct = (F_g_nProduct.to('newtons'))

print(_('F_g_wProduct','F_g_nProduct'))

F_g_wProduct = -11.358625905864681 newton
F_g_nProduct = -23.628877521741135 newton


<h1>Summary Of Linear Forces</h1>

<p>There is one particularly tricky part of this anaylsis, which depends on if the acceleration do to change in velocity is greater than acceleration due to gravity, and the direction of the acceleration.  Put simply on Side<sub>c</sub> the force of gravity can either work with or against the load.  Also, the mass there can include the product (prior to ejection) or not include the product (after ejection).  So, we must choose the timing and scenerio carefully to select the greatest load for the given acceleration</p>

4 Scenerios:
  <ul>
    <li>Motor is driving and aceleration due to motor is greater than gravity.</li>
    <li>Motor is breaking and acceleratioin due to break is greater than gravity.</li>
    <li>Motor is driving and acceleraiton due to motor is less than gravity.</li>
    <li>Motor is driving and acceleration due to break is less than gravity.</li>
  </ul>

<h2>Acceration 1 Load</h2>

<table style="border:none; border-collapse:collapse">
<tr>
  <td style="border:none"><img src="./assets/ChainSystemMotionProfileAcceleration1.svg"></td>
  <td style="border:none"><img src="./assets/ChainSystemLoadA.svg"></td>
<tr>
</table>

<table>

<p>Machine gets up to full speed from a start in 10s</p>

<i><u>Metrics</u></i> <br>
V<sub>0</sub> = 0"/s <br>
$V = V_run$ = 2.5"/s <br>

<i><u>Calcs</u></i><br>
$V = V_0 + at$ <br>
$a_1 = \frac{V - V_0}{t}$ <br>

<p>Force by sides</p>

<ul style="list-style-type: none">
    <li>
      Side A:
      <ul style="list-style-type: none">
        <li>Always full of product</li>
      </ul>
    </li>
    <li>
      Side B:
      <ul style="list-style-type: none">
        <li>Is the return path, never full of product </li>
      </ul>
    </li>
    <li>
      Side C:
      <ul style="list-style-type: none">
        <li>Gravity will assist this acceleration, and the calculated acceleration is less than gravity so we must select the scenerio where mass is minimized on side c.  This will be after the product has been ejected.</li>
      </ul>
    </li>
</ul>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a1} = m_{side\_a}a_1 + F_{friction}$<br>
$F_{b\_a1} = m_{side\_b}a_1 + F_{friction}$<br>
$F_{c\_a1} = m_{side\_c}a_1 + F_{friction}$<br>

$F_{total\_a1} = F_{a\_a1} + F_{b\_a1} + F_{c\_a1} + F_{g\_NoProduct} + 3(F_{friction})$

In [103]:
#Metrics
V_0 = 0
V = V_run
t = 10 * ureg.seconds

m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch) * Side_c

#Calcs
a_1 = (V - V_0) / t
F_a_a1 = m_side_a * a_1
F_b_a1 = m_side_b * a_1
F_c_a1 = m_side_c * a_1
F_total_a1 = F_a_a1 + F_b_a1 + F_c_a1 + abs(F_g_nProduct) + 3*F_friction

#Normalize
F_total_a1 = (F_total_a1.to('newtons'))

print(_('a_1','F_a_a1','F_b_a1','F_c_a1','F_total_a1'))

a_1 = 0.25 inch / second ** 2
F_a_a1 = 1.3923521733417157 inch * kilogram / second ** 2
F_b_a1 = 0.8363052732097894 inch * kilogram / second ** 2
F_c_a1 = 1.0277738374543206 inch * kilogram / second ** 2
F_total_a1 = 23.71159087635488 newton


<h2>Acceration 2 Load</h2>
 
<table style="border:none; border-collapse:collapse">
<tr>
  <td style="border:none"><img src="./assets/ChainSystemMotionProfileAcceleration2.svg"></td>
  <td style="border:none"><img src="./assets/ChainSystemLoadB.svg"></td>
<tr>
</table>

<table>

<p>Machine skips two pads every thirty to create a space.  If we look at the acceleration curve, we can see it is symetrical.  So, if we solve for one side, we can flip the sign for the other side.  The goal here is to skip two pads, so we need to go the distance of three.  Thats the two that are skipped, as well as the distance the last pad that got loaded (number 30) needs to travel.</p>

<i><u>Metrics</u></i> <br>

$\Delta d = \frac{.5" * 3}{2} = .75"\\$ <br>
$t = \frac{SecondsPerPad}{2} = .1s$<br><br>
V<sub>0</sub> = V<sub>run</sub> = 2.5"/s <br>

<br><i><u>Calcs</u></i><br>
$\Delta d = V_0t + \frac{1}{2}at^2 \\$<br>
$\frac{1}{2}at^2 = \Delta d - V_0t \\$<br>
$a_{2\_first} = \frac{2(\Delta d - V_0t)}{t^2} \\$<br>
$a_{2\_second} = -1 * \frac{2(\Delta d - V_0t)}{t^2} \\$<br>

<p>Force by sides</p>

<ul style="list-style-type: none">
    <li>
      Side A:
      <ul style="list-style-type: none">
        <li>Always full of product</li>
      </ul>
    </li>
    <li>
      Side B:
      <ul style="list-style-type: none">
        <li>Is the return path, never full of product </li>
      </ul>
    </li>
    <li>
      Side C:
      <ul style="list-style-type: none">
        <li>Gravity will assist this acceleration, and the calculated acceleration is less than gravity so we must select the scenerio where mass is minimized on side c.  This will be after the product has been ejected.</li>
      </ul>
    </li>
</ul>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a2\_first} = m_{side\_a}a_{2\_first} + F_{friction}$<br>
$F_{b\_a2\_first} = m_{side\_b}a_{2\_first} + F_{friction}$<br>
$F_{c\_a2\_first} = m_{side\_c}a_{2\_first} + F_{friction}$<br>

$F_{total\_a2\_first} = F_{a\_a2\_first} + F_{b\_a2\_first} + F_{c\_a2\_first}+ F_{g\_nProduct} + 3(F_{friction})$

In [104]:
#Metrics
dd = .75 * ureg.inches
t = .1 * ureg.seconds
V_0 = V_run

m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch) * Side_c

#Calcs
a_2_first = 2*(dd - V_0*t) / t**2
F_a_a2_first = m_side_a * a_2_first
F_b_a2_first = m_side_b * a_2_first
F_c_a2_first = m_side_c * a_2_first
F_total_a2_first = F_a_a2_first + F_b_a2_first + F_c_a2_first + abs(F_g_nProduct) + 3*F_friction

#Normalize
F_total_a2_first = (F_total_a2_first.to('newtons'))

print(_('a_2_first','F_a_a2_first','F_b_a2_first','F_c_a2_first','F_total_a2_first'))

a_2_first = 99.99999999999999 inch / second ** 2
F_a_a2_first = 556.9408693366862 inch * kilogram / second ** 2
F_b_a2_first = 334.5221092839157 inch * kilogram / second ** 2
F_c_a2_first = 411.1095349817282 inch * kilogram / second ** 2
F_total_a2_first = 56.71421936724031 newton


<p>When breaking, (the second half of the acceleration cure) the force of gravity on Side<sub>c</sub> is going against the breaking load, so it must maximize the force by using the greatest mass.</p>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch * productMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$a_{2\_second} = -a_{2\_first}$

$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a2\_second} = m_{side\_a}*a_{2\_second} + F_{friction}$<br>
$F_{b\_a2\_second} = m_{side\_b}*a_{2\_second} + F_{friction}$<br>
$F_{c\_a2\_second} = m_{side\_c}*a_{2\_second} + F_{friction}$<br>

$F_{total\_a2\_second} = F_{a\_a2\_second} + F_{b\_a2\_second} + F_{c\_a2\_second} + F_{g\_wProduct}+ 3(F_{friction})$

In [105]:
#Metrics
m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c

#Calcs
a_2_second = -1 * a_2_first
F_a_a2_second = m_side_a * a_2_second
F_b_a2_second = m_side_b * a_2_second
F_c_a2_second = m_side_c * a_2_second
F_total_a2_second = F_a_a2_second + F_b_a2_second + F_c_a2_second + abs(F_g_wProduct) + 3*F_friction

#Normalize
F_total_a2_second = (F_total_a2_second.to('newtons'))

print(_('a_2_second','F_a_a2_second','F_b_a2_second','F_c_a2_second','F_total_a2_second'))

a_2_second = -99.99999999999999 inch / second ** 2
F_a_a2_second = -556.9408693366862 inch * kilogram / second ** 2
F_b_a2_second = -334.5221092839157 inch * kilogram / second ** 2
F_c_a2_second = -536.1885524728235 inch * kilogram / second ** 2
F_total_a2_second = -24.903722983908327 newton


<h2>Acceration 3 Load</h2>

<table style="border:none; border-collapse:collapse">
<tr>
  <td style="border:none"><img src="./assets/ChainSystemMotionProfileAcceleration3.svg"></td>
  <td style="border:none"><img src="./assets/ChainSystemLoadC.svg"></td>
<tr>
</table>

<table>

<p>In the future we plan to integrate vision technology that automates our quality control, as well as automated packing equipement.  All such future integrations require a cull system that stops the stacker wheel when a bad pad is detected.  We should have some notice time to stop the wheel, as detection will happen prior to the wheel.  As such, we should only ever have to do a sudden stop from constant 2.5"/s velocity.  We will need to stop in .1s however, so that whatever stage the current pad load in the wheel is, we can catch it.</p>

<i><u>Metrics</u></i><br>
V<sub>0</sub> = V<sub>run</sub> = 2.5"/s<br>
t = SecondsPerPad / 2 = .2s / 2 = .1s<br>
V = 0"/s<br>

<i><u>Calcs</u></i><br>
$V = V_0 + at$ <br>
$a_3 = \frac{V - V_0}{t}$ <br>

<p>Force by sides</p>

<ul style="list-style-type: none">
    <li>
      Side A:
      <ul style="list-style-type: none">
        <li>Always full of product</li>
      </ul>
    </li>
    <li>
      Side B:
      <ul style="list-style-type: none">
        <li>Is the return path, never full of product </li>
      </ul>
    </li>
    <li>
      Side C:
      <ul style="list-style-type: none">
        <li>The force of gravity on Side<sub>c</sub> is going against the breaking load, so it must included with the greatest mass to maxiimize load on Side<sub>c</sub> .  This will be before the product has been ejected.</li>
      </ul>
    </li>
</ul>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a3} = m_{side\_a}a_{3} + F_{friction}$<br>
$F_{b\_a3} = m_{side\_b}a_{3} + F_{friction}$<br>
$F_{c\_a3} = m_{side\_c}a_{3} + F_{friction}$<br>

$F_{total\_a3} =  F_{a\_a3} + F_{b\_a3} + F_{c\_a3} + F_{g\_wProduct} + 3(F_{friction})$

In [106]:
#Metrics
V_0 = V_run
V = 0 * (ureg.inches / ureg.seconds)
t = .1 * ureg.seconds

m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c

#Calcs
a_3 = (V - V_0)/t

F_a_a3 = m_side_a * a_3
F_b_a3 = m_side_b * a_3
F_c_a3 = m_side_c * a_3
F_total_a3 = F_a_a3 + F_b_a3 + F_c_a3 + abs(F_g_wProduct) + 3*F_friction

#Normalize
F_total_a3 = (F_total_a3.to('newtons'))
F_total_a3 = 0 if F_total_a3 > 0 else F_total_a3

print(_('a_3','F_a_a3','F_b_a3','F_c_a3','F_total_a3'))

a_3 = -25.0 inch / second ** 2
F_a_a3 = -139.23521733417158 inch * kilogram / second ** 2
F_b_a3 = -83.63052732097894 inch * kilogram / second ** 2
F_c_a3 = -134.0471381182059 inch * kilogram / second ** 2
F_total_a3 = 0


<h2>Acceration 4 Load</h2>

<table style="border:none; border-collapse:collapse">
<tr>
  <td style="border:none"><img src="./assets/ChainSystemMotionProfileAcceleration4.svg"></td>
  <td style="border:none"><img src="./assets/ChainSystemLoadC.svg"></td>
<tr>
</table>

<table>

<p>This is simply the inverse calculation of Acceleration 3.  THis is for when the pads begin to pass QA again.</p>

<i><u>Metrics</u></i><br>
V<sub>0</sub> = 0"/s<br>
V = V<sub>run</sub> = 2.5"/s<br>
t = SecondsPerPad = .1s<br>

<i><u>Calcs</u></i><br>
$V = V_0 + at$ <br>
$a_4 = \frac{V - V_0}{t}$ <br>

<p>Force by sides</p>

<ul style="list-style-type: none">
    <li>
      Side A:
      <ul style="list-style-type: none">
        <li>Always full of product</li>
      </ul>
    </li>
    <li>
      Side B:
      <ul style="list-style-type: none">
        <li>Is the return path, never full of product </li>
      </ul>
    </li>
    <li>
      Side C:
      <ul style="list-style-type: none">
        <li>The force of gravity is going with the breaking load, so it must be minimized.  This will be after the product has been ejected.</li>
      </ul>
    </li>
</ul>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a4} = m_{side\_a}a_{4} + F_{friction}$<br>
$F_{b\_a4} = m_{side\_b}a_{4} + F_{friction}$<br>
$F_{c\_a4} = m_{side\_c}a_{4} + F_{friction}$<br>

$F_{total\_a4} = F_{a\_a4} + F_{b\_a4}+ F_{c\_a4} + F_{g\_nProduct} + 3(F_{friction})$

In [107]:
#Metrics
V_0 = 0 * (ureg.inches / ureg.seconds)
V = V_run
t = .1 * ureg.seconds

m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch) * Side_c

#Calcs
a_4 = (V - V_0)/t

F_a_a4 = m_side_a * a_4
F_b_a4 = m_side_b * a_4
F_c_a4 = m_side_c * a_4
F_total_a4 = F_a_a4 + F_b_a4 + F_c_a4 + abs(F_g_nProduct) + 3*F_friction

#Normalize
F_total_a4 = (F_total_a4.to('newtons'))

print(_('a_4','F_a_a4','F_b_a4','F_c_a4','F_total_a4'))

a_4 = 25.0 inch / second ** 2
F_a_a4 = 139.23521733417158 inch * kilogram / second ** 2
F_b_a4 = 83.63052732097894 inch * kilogram / second ** 2
F_c_a4 = 102.77738374543206 inch * kilogram / second ** 2
F_total_a4 = 31.90021298311593 newton


<h2>Acceration 5 Load</h2>

<table style="border:none; border-collapse:collapse">
<tr>
  <td style="border:none"><img src="./assets/ChainSystemMotionProfileAcceleration5.svg"></td>
  <td style="border:none"><img src="./assets/ChainSystemLoadC.svg"></td>
<tr>
</table>

<table>

<p>Estop.  Load must stop within 5s.</p>

<i><u>Metrics</u></i><br>
V<sub>0</sub> = V<sub>run</sub> = 2.5"/s<br>
V = 0"/s<br>
t = 5s<br>

<i><u>Calcs</u></i><br>
$V = V_0 + at$ <br>
$a_5 = \frac{V - V_0}{t}$ <br>

<p>Force by sides</p>

<ul style="list-style-type: none">
    <li>
      Side A:
      <ul style="list-style-type: none">
        <li>Always full of product</li>
      </ul>
    </li>
    <li>
      Side B:
      <ul style="list-style-type: none">
        <li>Is the return path, never full of product </li>
      </ul>
    </li>
    <li>
      Side C:
      <ul style="list-style-type: none">
        <li>The force of gravity is going against the breaking load, so it must be maximized.  This will be before the product has been ejected.</li>
      </ul>
    </li>
</ul>

<i><u>Metrics</u></i> <br>
$m_{side\_a} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a$ <br>
$m_{side\_b} = (chainMassPerInch + paddleMassPerInch) * Side_b$ <br>
$m_{side\_c} = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c$ <br>


<i><u>Calcs</u></i><br>
$F_{total} = F{a} + F{b} + F{c}$<br>
$F_{a\_a5} = m_{side\_a}a_{5} + F_{friction}$<br>
$F_{b\_a5} = m_{side\_b}a_{5} + F_{friction}$<br>
$F_{c\_a5} = m_{side\_c}a_{5} + F_{friction}$<br>

$F_{total\_a5} = F_{a\_a5} + F_{b\_a5}+ F_{c\_a5} + F_{g\_wProduct} + 3(F_{friction})$

In [108]:
#Metrics
V_0 = V_run
V = 0 * (ureg.inches / ureg.seconds)
t = 5 * ureg.seconds

m_side_a = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_a
m_side_b = (chainMassPerInch + paddleMassPerInch) * Side_b
m_side_c = (chainMassPerInch + paddleMassPerInch + productMassPerInch) * Side_c

#Calcs
a_5 = (V - V_0)/t

F_a_a5 = m_side_a * a_5
F_b_a5 = m_side_b * a_5
F_c_a5 = m_side_c * a_5
F_total_a5 = F_a_a5 + F_b_a5 + F_c_a5 + abs(F_g_wProduct) + 3*F_friction

#Normalize
F_total_a5 = (F_total_a5.to('newtons'))
F_total_a5 = 0 if F_total_a5 > 0 else F_total_a5

print(_('a_5','F_a_a5','F_b_a5','F_c_a5','F_total_a5'))

a_5 = -0.5 inch / second ** 2
F_a_a5 = -2.7847043466834314 inch * kilogram / second ** 2
F_b_a5 = -1.6726105464195788 inch * kilogram / second ** 2
F_c_a5 = -2.680942762364118 inch * kilogram / second ** 2
F_total_a5 = 0


<h3>Summary of forces</h3>

In [109]:
print(_('F_total_a1', 'F_total_a2_first', 'F_total_a2_second', 'F_total_a3', 'F_total_a4', 'F_total_a5'))

F_total_a1 = 23.71159087635488 newton
F_total_a2_first = 56.71421936724031 newton
F_total_a2_second = -24.903722983908327 newton
F_total_a3 = 0
F_total_a4 = 31.90021298311593 newton
F_total_a5 = 0


<h1>Inertia Calculations</h1>

<img src="./assets/ChainSystemInertiaBasic.svg">

<p>Looking at all the acceleration calcs, it seems clear our higest load is on the first part of Acceleration<sub>3</sub> .  For that reason let's look at that load from now on.</p>

F<sub>load</sub> = F<sub>total_a2_first</sub> = 56.71421936724031 N

<p>Next we'll need the metrics for all our pully's sprockets and shafts:</p>

<ul>
<li>
    <b>Drive Sprocket</b>
    <ul>
      <li>46 teeth</li>
      <li>1.25" bore</li>
      <li>11.5 lbs</li>
      <li>7.61" OD</li>
      <li>7.165605095541401" OD</li>
      <li>2 counts</li>
    </ul>
  </li>
  <li>
    <b>Idler Sprocket</b>
    <ul>
      <li>21 teeth</li>
      <li>1.25" bore</li>
      <li>2.6 lbs</li>
      <li>3.46" OD</li>
      <li>4 counts</li>
    </ul>
  </li>
  <li>
    <b>Pulleys</b>
    <ul>
      <li>34 8M 30</li>
      <li>1.25" bore</li>
      <li>2.4 lbs</li>
      <li>3.355" OD</li>
      <li>2 counts</li>
    </ul>
  </li>
  <li>
    <b>Pulley SH Bushins</b>
    <ul>
      <li>1.25" bore</li>
      <li>1 lbs</li>
      <li>2 counts</li>
    </ul>
  </li>
  <li>
    <b>Shafting</b>
    <ul>
      <li>1.25" radius</li>
      <li>4.81 lbs/ft</li>
      <li>12" for drive sprocket</li>
      <li>8" x 2 for idle sprockets</li>
      <li>3" for pulley to gearbox</li>
      <li></li>
    </ul>
  </li>
  <li>
    <b>Pully Belt</b>
    <ul>
      <li>9/60 8M 30</li>
      <li>.4 lbs</li>
    </ul>
  </li>
</ul>

<h2><b>Simple Inertia of All Sprockets and pullyes<b></h2>
<p>To be reasonable with our estimate, I'm going to assume that everything is a solid disk.  This won't be perfect, but should be fairly accurate.</p>

<i><u>Calcs</u></i><br>
$I = \frac{1}{2}mr^2$<br>

<i><u>Global Metrics</u></i><br>
ShaftingMassPerInch = 4.81lbs / 12"<br>
m<sub>DriveSprocket</sub> = MassOfSprocket + ShaftingMassPerInch * 1"<br>
m<sub>IdlerSprocket</sub> = MassOfIdler + ShaftingMassPerInch * 1"<br>
m<sub>Pully</sub> = MassOfIdler + MassOfBushing + ShaftingMassPerInch * 1"<br>


In [110]:
#Metrics
ShaftingMassPerInch = (4.81 * ureg.lbs) / (12 * ureg.inches)

m_DriveSprocket = 11.5 * ureg.lbs + (ShaftingMassPerInch * ureg.inches)
r_DriveSprocket = (7.165 * ureg.inches) / 2

m_IdlerSprocket = 2.6 * ureg.lbs + (ShaftingMassPerInch * ureg.inches)
r_IdlerSprocket = (3.46 * ureg.inches) / 2

m_Pulley = (2.4 * ureg.lbs) + (1 * ureg.lbs) + (ShaftingMassPerInch * ureg.inches)
r_Pulley = (3.355 * ureg.inches) / 2

l_ShaftingTotalLength = (12 + 8*2 + 3 ) * ureg.inches #Guess for now
r_Shafting = (1.25 * ureg.inches) / 2

#Calcs
I_DriveSprocket = .5 * m_DriveSprocket * r_DriveSprocket**2
I_IdlerSprocket = .5 * m_IdlerSprocket * r_IdlerSprocket**2
I_Pulley = .5 * m_Pulley * r_Pulley**2
I_Shafting = .5 * ShaftingMassPerInch * l_ShaftingTotalLength * r_Shafting**2

print(_('I_DriveSprocket', 'I_IdlerSprocket', 'I_Pulleys', 'I_Shafting'))

I_DriveSprocket = 76.36946981510417 inch ** 2 * pound
I_IdlerSprocket = 4.490597041666667 inch ** 2 * pound
I_Pulleys = 5.3477843776041665 inch ** 2 * pound
I_Shafting = 2.4269205729166665 inch ** 2 * pound


<h2><b>Load Inertia</b></h2>

<p> Now to get out peak acceleration we know we need this much force, which includes the gravitational force that must be overcome.<p>

F<sub>load</sub> = F<sub>total_a2_first</sub> = 56.71421936724031 N

<p>Now, F<sub>load</sub> can be considered the effective force of the stacker wheel load.  And we know the peak acceration that was used to calculate it.  So if we look at the system now from the Drive sprockets point of view</p>

$a = a_{2\_first}\\$
$F_{load} = F_{total_a2\_first}$ <br>

$F_{load} * r_{DriveSprocket} = \Tau\\$ <br>
$\alpha = \frac{a}{r_{DriveSprocket}}\\$ <br>

<i><u>Calcs</u></i><br>
$F_{load} * r_{DriveSprocket} = I * \alpha\\$ <br>
$F_{load} * r_{DriveSprocket} = I * \frac{a}{r_{DriveSprocket}}\\$ <br>
$I = (F_{load} * r_{DriveSprocket})*\frac{r_{DriveSprocket}}{a}\\$ <br>
$I = \frac{(F_{load} * r_{DriveSprocket}^2)}{a_{2\_first}}\\$ <br>



In [111]:
r = r_DriveSprocket.to('meters')
a = a_2_first.to('meters/seconds**2')
F = F_total_a2_first.to('kilogram*meters/seconds**2')
I_load = (F*r**2) / a
print(_('I_load'))

I_load = 0.18488346566256622 kilogram * meter ** 2


<h2><b>System Inertia</b></h2>

$I_{system} = 2I_{DriveSprocket} + 4I_{IdlerSprocket} + 2I_{Pulley} + I_{Shafting} + I_{Load}$

In [112]:
I_DriveSprocket_Normalized = I_DriveSprocket.to('kilogram*meter**2')
I_IdlerSprocket_Normalized = I_IdlerSprocket.to('kilogram*meter**2')
I_Pulley_Normalized = I_Pulley.to('kilogram*meter**2')
I_Shafting_Normalized = I_Shafting.to('kilogram*meter**2')

I_System = 2*I_DriveSprocket_Normalized + 4*I_IdlerSprocket_Normalized + 2*I_Pulley_Normalized + I_Shafting_Normalized + I_load

print(_('I_System', 'I_DriveSprocket_Normalized', 'I_IdlerSprocket_Normalized', 'I_Pulley_Normalized', 'I_Shafting_Normalized'))

I_System = 0.23867760379805886 kilogram * meter ** 2
I_DriveSprocket_Normalized = 0.022348735179263834 kilogram * meter ** 2
I_IdlerSprocket_Normalized = 0.001314126761963524 kilogram * meter ** 2
I_Pulley_Normalized = 0.0015649737668761733 kilogram * meter ** 2
I_Shafting_Normalized = 0.0007102131953585287 kilogram * meter ** 2


<h2><b>Drive Speed</b></h2>

<p>Now we need to look at the velocity's and get angular drive speeds.  We should have three of these to concern ourselves with:</p>
<ul>
<li>Stopped</li>
<li>Constant 30 pad velocity</li>
<li>Peak velocity during skip</li>
</ul>

<i><u>Calcs</u></i><br>
$V_{stop} = 0m/s$<br>
$V_{run} = V_run$<br>

$V_0 = V_run$<br>
$a = a_{a2\_first}$<br>
$t = .2s / 2$<br>
$V_{peak} = V_0 + at$<br>

<p>We will then need to calculate the rotational velocity at the drive.<p>

$V = r\omega$

In [113]:
V_stop = 0 * (ureg.inches / ureg.seconds)
V_peak = V_run + a_2_first * (.1 *  ureg.seconds)

w_stop = V_stop / r_DriveSprocket
w_run = V_run / r_DriveSprocket
w_peak = V_peak / r_DriveSprocket

w_stop_rpm = w_stop.to('rpm')
w_run_rpm = w_run.to('rpm')
w_peak_rpm = w_peak.to('rpm')

print(_('V_stop','V_run','V_peak','w_stop_rpm','w_run_rpm','w_peak_rpm'))

V_stop = 0 inch / second
V_run = 2.5 inch / second
V_peak = 12.5 inch / second
w_stop_rpm = 0.0 revolutions_per_minute
w_run_rpm = 6.663849675864425 revolutions_per_minute
w_peak_rpm = 33.319248379322126 revolutions_per_minute
